# SciPy Interpolation

1D interpolation, splines, multidimensional interpolation, and scattered-data interpolation with practical examples.

In [1]:
import numpy as np
from scipy.interpolate import interp1d, CubicSpline, PchipInterpolator, UnivariateSpline, RegularGridInterpolator, Rbf, griddata

np.set_printoptions(precision=4, suppress=True)

## 1. One-Dimensional Interpolation
1D interpolation fills missing values and estimates values between measurements.

In [2]:
hours = np.array([0, 3, 6, 9, 12, 15, 18, 21, 24], dtype=float)
temp = np.array([14, 13, 15, 19, 24, 27, 23, 18, 16], dtype=float)
query = np.array([2, 5, 11, 17, 22], dtype=float)

linear = interp1d(hours, temp, kind='linear')(query)
cubic = interp1d(hours, temp, kind='cubic')(query)

print("Example 1: Hourly temperature completion")
print("Linear estimates:", np.round(linear, 2))
print("Cubic estimates:", np.round(cubic, 2))

maturity = np.array([1, 2, 3, 5, 10], dtype=float)
yields = np.array([3.1, 3.3, 3.5, 3.8, 4.1], dtype=float)
curve = PchipInterpolator(maturity, yields)

print()
print("Example 2: Yield curve estimation")
print(f"4-year yield: {curve(4.0):.3f}")
print(f"7-year yield: {curve(7.0):.3f}")

Example 1: Hourly temperature completion
Linear estimates: [13.33 14.33 22.33 24.33 17.33]
Cubic estimates: [12.95 14.06 22.32 24.83 16.87]

Example 2: Yield curve estimation
4-year yield: 3.671
7-year yield: 3.971


## 2. Spline Modeling
Splines produce smooth curves and derivatives for trend and rate analysis.

In [3]:
time = np.array([0, 2, 4, 6, 8, 10], dtype=float)
position = np.array([0, 8, 23, 41, 62, 85], dtype=float)
traj = CubicSpline(time, position)

print("Example 1: Vehicle trajectory")
print(f"Position at t=5: {traj(5.0):.3f}")
print(f"Velocity at t=5: {traj(5.0, 1):.3f}")
print(f"Acceleration at t=5: {traj(5.0, 2):.3f}")

rng = np.random.default_rng(5)
day = np.arange(1, 61)
quality_true = 92 + 0.03 * day + 0.8 * np.sin(day / 5)
quality_obs = quality_true + rng.normal(0, 0.6, size=day.size)
quality_spline = UnivariateSpline(day, quality_obs, s=25)

print()
print("Example 2: Manufacturing quality trend")
print(f"Smoothed quality day 30: {quality_spline(30):.3f}")
print(f"Trend slope day 30: {quality_spline.derivative()(30):.4f}")

Example 1: Vehicle trajectory
Position at t=5: 31.663
Velocity at t=5: 8.965
Acceleration at t=5: 0.675

Example 2: Manufacturing quality trend
Smoothed quality day 30: 92.533
Trend slope day 30: 0.0613


## 3. Multidimensional Interpolation
Grid and surface interpolation estimates values across 2D parameter spaces.

In [4]:
temp_grid = np.array([10, 20, 30, 40], dtype=float)
load_grid = np.array([0.2, 0.5, 0.8, 1.0], dtype=float)
efficiency = np.array([
    [0.82, 0.84, 0.81, 0.78],
    [0.85, 0.88, 0.86, 0.83],
    [0.83, 0.87, 0.89, 0.87],
    [0.79, 0.84, 0.86, 0.85]
])

interp_eff = RegularGridInterpolator((temp_grid, load_grid), efficiency)
q = np.array([[25, 0.65], [34, 0.9]])

print("Example 1: Battery efficiency map")
print("Interpolated values:", np.round(interp_eff(q), 4))

rng = np.random.default_rng(44)
xy = rng.uniform(0, 10, size=(35, 2))
aqi = 40 + 2 * np.sin(xy[:, 0] / 2) + 3 * np.cos(xy[:, 1] / 3)
aqi_est = griddata(xy, aqi, np.array([[4.5, 6.0], [8.0, 2.0]]), method='cubic')

print()
print("Example 2: Air-quality interpolation")
print("Estimated AQI:", np.round(aqi_est, 3))

Example 1: Battery efficiency map
Interpolated values: [0.875 0.87 ]

Example 2: Air-quality interpolation
Estimated AQI: [40.311 40.806]


## 4. Scattered Data with Radial Basis Functions
RBF interpolation handles irregular sensor layouts where grid methods are not suitable.

In [5]:
rng = np.random.default_rng(61)

x = rng.uniform(0, 30, size=80)
y = rng.uniform(0, 20, size=80)
strength = -40 - 0.6 * x - 0.3 * y + 2 * np.sin(x / 5) + rng.normal(0, 0.8, size=80)
rbf_wifi = Rbf(x, y, strength, function='multiquadric', smooth=1.5)

query = np.array([[10, 8], [18, 14], [25, 5]], dtype=float)

print("Example 1: WiFi strength mapping")
print("Estimated strengths:", np.round(rbf_wifi(query[:, 0], query[:, 1]), 2))

x2 = rng.uniform(-5, 5, size=60)
y2 = rng.uniform(-5, 5, size=60)
water = 18 - 0.4 * (x2 ** 2 + 0.6 * y2 ** 2) + rng.normal(0, 0.3, size=60)
rbf_water = Rbf(x2, y2, water, function='gaussian', epsilon=1.6)

print()
print("Example 2: Groundwater level estimation")
print(f"Level at (1.5, -2.0): {rbf_water(1.5, -2.0):.3f}")
print(f"Level at (-3.0, 2.5): {rbf_water(-3.0, 2.5):.3f}")

Example 1: WiFi strength mapping
Estimated strengths: [-46.52 -56.29 -57.74]

Example 2: Groundwater level estimation
Level at (1.5, -2.0): 13.170
Level at (-3.0, 2.5): 12.465
